In [1]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
def entropy(y):
    class_count = np.bincount(y)
    prob = [i/len(y) for i in class_count]
    entropy = -np.sum([p * np.log(p) for p in prob if p > 0])
    return entropy

In [3]:
class Node:
    def __init__(self,feature = None, threshold = None, left = None, right = None, value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf_node(self):
        return self.value is not None

In [4]:
class DecisionTree:
    def __init__(self,min_samples=3,max_depth=100,n_features = None):
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
    
    def fit(self,X,y):
        if not self.n_features:
            self.n_features = X.shape[1]
        else:
            self.n_features = min(self.n_features,X.shape[1])
        
        self.root = self.grow_tree(X,y)
            
    def grow_tree(self,X,y,depth=0):
        n_samples,n_feats = X.shape
        n_labels = len(np.unique(y))
        
        # Leaf Node Condition
        
        if(depth == self.max_depth or n_samples < self.min_samples or n_labels == 1):
            leaf_node = self.most_common_label(y)
            return Node(value=leaf_node)
        
        # Select Feature Index at random
        feat_idx = np.random.choice(n_feats,self.n_features,replace=False)
        best_idx,best_thresh = self.best_criteria(X,y,feat_idx)
        
        # Find Left and Right Tree
        left_idxs, right_idxs = self.split_data(best_thresh,X[:, best_idx])
        left = self.grow_tree(X[left_idxs,:],y[left_idxs],depth+1)
        right = self.grow_tree(X[right_idxs,:],y[right_idxs],depth+1)
        return Node(best_idx,best_thresh,left,right)
    
    # Find Majority Vote for label in samples of data
    def most_common_label(self,y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    # find best feature and best threshold
    def best_criteria(self,X,y,feat_idx):
        best_gain = 0 # Best gain from All Features
        best_idx = None # Best Feature Index
        best_thresh = None # Best threshold to split the Feature
        
        # Iterate through each feature one by one and find the best threshold
        for feat in feat_idx:
            feat_samples = X[:,feat]
            un_thresh = np.unique(feat_samples)
            
            # Iterate through each unique value of feature
            for thresh in un_thresh:
                gain = self.information_gain(y,feat_samples,thresh)
                
                if gain > best_gain:
                    best_thresh = thresh
                    best_idx = feat
                    best_gain = gain
                    
        return best_idx,best_thresh
    
    # Information Gain
    def information_gain(self,y,feat_samples,thresh):
        parent_entropy = entropy(y)
        
        # Split the samples into left and right indexes
        left_idxs, right_idxs = self.split_data(thresh,feat_samples)
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig
    
    def split_data(self,split_thresh,X_column):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs,right_idxs
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

## Random Forest

In [5]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [71]:
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [62]:
def bootstrap_samples(X,y):
    n_samples = X.shape[0]
    indexes = np.random.choice(n_samples,n_samples,replace=True)
    return X[indexes],y[indexes]

In [53]:
class RandomForest:
    def __init__(self,n_trees = 3,min_samples = 3, max_depth = 10,n_features = None):
        self.n_trees = n_trees
        self.n_features = n_features
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.trees = []
    
    def fit(self,X,y):
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples = self.min_samples,
                max_depth = self.max_depth,
                n_features = self.n_features
            )
            X_samples,y_samples = bootstrap_samples(X,y)
            tree.fit(X_samples,y_samples)
            self.trees.append(tree)
    
    def predict(self,X):
        y_pred_tree = [t.predict(X) for t in self.trees]
        y_pred_tree = np.swapaxes(y_pred_tree,1,0) # Change Axes to compute majority vote
        y_pred = [most_common_label(y) for y in y_pred_tree]
        return np.array(y_pred)

In [54]:
random_forest = RandomForest(n_trees = 10,min_samples = 3, max_depth = 10,n_features = 30)

In [55]:
random_forest.fit(X_train,y_train)

In [56]:
y_pred = random_forest.predict(X_test)

In [57]:
y_pred.shape

(114,)

In [58]:
accuracy = np.sum(y_test == y_pred) / len(y_test)
accuracy

0.9298245614035088